In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint



# 数据准备
def load_images(df, path_prefix):
    image_list = []
    for idx, row in df.iterrows():
        img_path = os.path.join(path_prefix, row['filepaths'])
        img = Image.open(img_path).resize((224, 224))
        img_array = np.array(img) / 255.0
        image_list.append(img_array)
    return np.array(image_list)

# 读取CSV文件和图片
df = pd.read_csv('/content/drive/MyDrive/archive/dogs.csv')
df_selected = df[df['labels'].isin(["Airedale", "Beagle", "Bloodhound", "Bluetick", "Chihuahua",
                                    "Collie", "Dingo", "French Bulldog", "German Shepherd",
                                    "Malinois", "Newfoundland", "Pekinese", "Pomeranian", "Pug", "Vizsla"])]
images = load_images(df_selected, '/content/drive/MyDrive/archive')
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_selected['labels'])

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建模型
def build_vgg16_model(input_shape, num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model = build_vgg16_model((224, 224, 3), len(np.unique(labels)))

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.0004),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

# 数据增强
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False)

# 设置ModelCheckpoint
checkpoint_path = "vgg16_best_model.h5"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only=True,
                             monitor='val_accuracy',
                             mode='max',
                             verbose=1)

# 训练模型，加入ModelCheckpoint回调
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=30,
          validation_data=(X_test, y_test),
          callbacks=[checkpoint])

# 评估模型
val_loss, val_acc = model.evaluate(X_test, y_test)
print(f"Validation Accuracy: {val_acc}")


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


48/48 [==============================] - ETA: 0s - loss: 2.4480 - sparse_categorical_accuracy: 0.2759 

48/48 [==============================] - 1162s 24s/step - loss: 2.4480 - sparse_categorical_accuracy: 0.2759 - val_loss: 1.3674 - val_sparse_categorical_accuracy: 0.6154
Epoch 2/30
48/48 [==============================] - ETA: 0s - loss: 1.4394 - sparse_categorical_accuracy: 0.5285 

48/48 [==============================] - 1196s 25s/step - loss: 1.4394 - sparse_categorical_accuracy: 0.5285 - val_loss: 0.9269 - val_sparse_categorical_accuracy: 0.7480
Epoch 3/30
48/48 [==============================] - ETA: 0s - loss: 1.0881 - sparse_categorical_accuracy: 0.6439 

48/48 [==============================] - 1185s 25s/step - loss: 1.0881 - sparse_categorical_accuracy: 0.6439 - val_loss: 0.7796 - val_sparse_categorical_accuracy: 0.7851
Epoch 4/30
 2/48 [>.............................] - ETA: 15:03 - loss: 0.9754 - sparse_categorical_accuracy: 0.7500

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet_v2.preprocess_input(img_array_expanded_dims)

def predict_and_evaluate(model, image_dir):
    correct_predictions = 0
    total_images = 0

    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                img_path = os.path.join(root, file)
                processed_img = load_and_preprocess_image(img_path)
                prediction = model.predict(processed_img)
                predicted_class = np.argmax(prediction, axis=1)[0]

                total_images += 1

    accuracy = correct_predictions / total_images if total_images > 0 else 0
    print(f"Accuracy: {accuracy:.2f}")

# 加载模型
model_path = 'vgg16_best_model.h5'
model = tf.keras.models.load_model(model_path)

# 验证数据集的路径
validation_images_path = '/content/drive/MyDrive/archive/valid'

# 进行预测并评估
predict_and_evaluate(model, validation_images_path)


In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import tensorflow as tf

# 指定圖片文件夾路徑
test_images_directory = '/content/drive/MyDrive/archive/Testing set'

# 加載模型
model = load_model('vgg16_best_model.h5', compile=False)

# 讀取圖片並進行預測
def predict_images_from_directory(directory):
    predicted_labels = []
    file_names = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        img = image.load_img(file_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array_expanded_dims = np.expand_dims(img_array, axis=0)
        img_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(img_array_expanded_dims)

        prediction = model.predict(img_preprocessed)
        predicted_class = np.argmax(prediction, axis=1)

        # 儲存文件名和預測結果
        file_names.append(file)
        predicted_labels.append(predicted_class[0])

    return file_names, predicted_labels

# 使用函數進行預測
file_names, predicted_classes = predict_images_from_directory(test_images_directory)

# 創建結果DataFrame
results_df = pd.DataFrame({
    'FileName': file_names,
    'PredictedClass': predicted_classes
})

# 將結果保存為Excel文件
results_df.to_excel('test_data.xlsx', index=False)
